# import packages and libraries

In [ ]:
import numpy as np
import keras
import tensorflow
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D, CenterCrop
from keras.models import Model
from keras.layers import concatenate,Dropout,Flatten
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

# uploading and preparing data

In [ ]:
# loading data
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
# data preprocessing
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
num_classes = 10
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

# AlexNet Dropout 0.5

In [ ]:
def alexnet(img_input,classes=10):
  xnet = Conv2D(96,(11,11),strides=(4,4),padding='same',activation='relu',kernel_initializer='uniform')(img_input)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Flatten()(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.5)(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.5)(xnet)
  out_model = Dense(classes, activation='softmax')(xnet)
  return out_model

In [ ]:
# show network structure
img_input=Input(shape=(32,32,3))
output = alexnet(img_input)
model1=Model(img_input,output)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 96)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 384)         885120

In [ ]:
# fitting the model
sgd = tensorflow.keras.optimizers.SGD(learning_rate=.01, momentum=0.9, nesterov=True) 
model1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
epochs = 120
batch_size = 128
iterations = int(x_train.shape[0]/batch_size)
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.115,height_shift_range=0.115, fill_mode='constant',cval=0.)
datagen.fit(x_train)
model1.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=iterations, epochs=epochs, callbacks=callbacks_list, validation_data=(x_test, y_test))

In [ ]:
# Generate accuracy plot
f, ax = plt.subplots()
ax.plot([None] + model1.history.history['accuracy'], 'o-')
ax.plot([None] + model1.history.history['val_accuracy'], 'x-')
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

In [ ]:
# Generate loss plot
f, ax = plt.subplots()
ax.plot([None] + model1.history.history['loss'], 'o-')
ax.plot([None] + model1.history.history['val_loss'], 'x-')
ax.legend(['Train loss', 'Validation loss'], loc = 0)
ax.set_title('Training/Validation loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('loss')

# AlexNet Dropout 0.25

In [ ]:
def alexnetV1(img_input,classes=10):
  xnet = Conv2D(96,(11,11),strides=(4,4),padding='same',activation='relu',kernel_initializer='uniform')(img_input)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Flatten()(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.25)(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.25)(xnet)
  out_model = Dense(classes, activation='softmax')(xnet)
  return out_model

In [ ]:
# show network structure
img_input=Input(shape=(32,32,3))
output = alexnetV1(img_input)
model2=Model(img_input,output)
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 96)          34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 96)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 2, 2, 384)         8851

In [ ]:
# fitting the model
sgd = tensorflow.keras.optimizers.SGD(learning_rate=.01, momentum=0.9, nesterov=True) 
model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
epochs = 120
batch_size = 128
iterations = int(x_train.shape[0]/batch_size)
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.115,height_shift_range=0.115, fill_mode='constant',cval=0.)
datagen.fit(x_train)
model2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=iterations, epochs=epochs, callbacks=callbacks_list, validation_data=(x_test, y_test))

In [ ]:
# Generate accuracy plot
f, ax = plt.subplots()
ax.plot([None] + model2.history.history['accuracy'], 'o-')
ax.plot([None] + model2.history.history['val_accuracy'], 'x-')
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

In [ ]:
# Generate loss plot
f, ax = plt.subplots()
ax.plot([None] + model2.history.history['loss'], 'o-')
ax.plot([None] + model2.history.history['val_loss'], 'x-')
ax.legend(['Train loss', 'Validation loss'], loc = 0)
ax.set_title('Training/Validation loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('loss')

# AlexNet Dropout 0.9

In [ ]:
# AlexNet Architecture
def alexnetV2(img_input,classes=10):
  xnet = Conv2D(96,(11,11),strides=(4,4),padding='same',activation='relu',kernel_initializer='uniform')(img_input)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform')(xnet)
  xnet = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(xnet)
  xnet = Flatten()(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.9)(xnet)
  xnet = Dense(4096,activation='relu')(xnet)
  xnet = Dropout(0.9)(xnet)
  out_model = Dense(classes, activation='softmax')(xnet)
  return out_model

In [ ]:
# show network structure
img_input=Input(shape=(32,32,3))
output = alexnetV2(img_input)
model3=Model(img_input,output)
model3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 96)          34944     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 4, 4, 96)         0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 256)         614656    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 2, 2, 384)         8851

In [ ]:
# Fitting the model
sgd = tensorflow.keras.optimizers.SGD(learning_rate=.01, momentum=0.9, nesterov=True) 
model3.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
epochs = 120
batch_size = 128
iterations = int(x_train.shape[0]/batch_size)
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.115,height_shift_range=0.115, fill_mode='constant',cval=0.)
datagen.fit(x_train)
model3.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=iterations, epochs=epochs, callbacks=callbacks_list, validation_data=(x_test, y_test))

In [ ]:
# Generate accuracy plot
f, ax = plt.subplots()
ax.plot([None] + model3.history.history['accuracy'], 'o-')
ax.plot([None] + model3.history.history['val_accuracy'], 'x-')
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

In [ ]:
# Generate loss plot
f, ax = plt.subplots()
ax.plot([None] + model3.history.history['loss'], 'o-')
ax.plot([None] + model3.history.history['val_loss'], 'x-')
ax.legend(['Train loss', 'Validation loss'], loc = 0)
ax.set_title('Training/Validation loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('loss')